<a href="https://colab.research.google.com/github/ChavesHygor/BairesDev---Machine-Learning-Training-ok/blob/main/objetos_similares.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#hide
from google.colab import drive
drive.mount('/content/drive')
#hide
%%writefile kaggle.json
{"username":"<your kaggle username>","key":"<your kaggle api key>"}
#hide
!pip install -q -U kaggle
!pip install --upgrade --force-reinstall --no-deps kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
#hide-output
# downloading raw images from kaggle
!kaggle datasets download -d paramaggarwal/fashion-product-images-small
!unzip fashion-product-images-small.zip

import pandas as pd
from shutil import move
import os
from tqdm import tqdm

# Criar e organizar diretórios
os.mkdir('/content/Fashion_data')
os.chdir('/content/Fashion_data')

df = pd.read_csv('/content/styles.csv', usecols=['id','masterCategory']).reset_index()
df['id'] = df['id'].astype('str')

all_images = os.listdir('/content/images/')
co = 0
os.mkdir('/content/Fashion_data/categories')
for image in tqdm(all_images):
    category = df[df['id'] == image.split('.')[0]]['masterCategory']
    category = str(list(category)[0])
    if not os.path.exists(os.path.join('/content/Fashion_data/categories', category)):
        os.mkdir(os.path.join('/content/Fashion_data/categories', category))
    path_from = os.path.join('/content/images', image)
    path_to = os.path.join('/content/Fashion_data/categories', category, image)
    move(path_from, path_to)
    co += 1
print('Moved {} images.'.format(co))

Mounted at /content/drive


UsageError: Line magic function `%%writefile` not found.


In [ ]:
import itertools
import os

import matplotlib.pylab as plt
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub

print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

MODULE_HANDLE = 'https://tfhub.dev/google/bit/m-r50x3/1'
IMAGE_SIZE = (224, 224)
print("Using {} with input size {}".format(MODULE_HANDLE, IMAGE_SIZE))
BATCH_SIZE = 32
N_FEATURES = 256

# Preparação dos geradores de dados
data_dir = '/content/Fashion_data/categories'
datagen_kwargs = dict(rescale=1./255, validation_split=.20)
dataflow_kwargs = dict(target_size=IMAGE_SIZE, batch_size=BATCH_SIZE,
                       interpolation="bilinear")

valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    **datagen_kwargs)
valid_generator = valid_datagen.flow_from_directory(
    data_dir, subset="validation", shuffle=False, **dataflow_kwargs)

do_data_augmentation = False
if do_data_augmentation:
  train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
      rotation_range=40,
      horizontal_flip=True,
      width_shift_range=0.2, height_shift_range=0.2,
      shear_range=0.2, zoom_range=0.2,
      **datagen_kwargs)
else:
  train_datagen = valid_datagen
train_generator = train_datagen.flow_from_directory(
    data_dir, subset="training", shuffle=True, **dataflow_kwargs)

# Construção do modelo de Deep Learning
print("Building model with", MODULE_HANDLE)
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=IMAGE_SIZE + (3,)),
    hub.KerasLayer(MODULE_HANDLE, trainable=False),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(N_FEATURES,
                           kernel_regularizer=tf.keras.regularizers.l2(0.0001)),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(train_generator.num_classes,
                           kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])
model.build((None,)+IMAGE_SIZE+(3,))
model.summary()

# Compilação e Treinamento do modelo
lr = 0.003 * BATCH_SIZE / 512
SCHEDULE_LENGTH = 500
SCHEDULE_BOUNDARIES = [200, 300, 400]
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries=SCHEDULE_BOUNDARIES,
                                                                   values=[lr, lr*0.1, lr*0.001, lr*0.0001])
optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9)
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimizer,
              loss=loss_fn,
              metrics=['accuracy'])
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = valid_generator.samples // valid_generator.batch_size
hist = model.fit(
    train_generator,
    epochs=5, steps_per_epoch=steps_per_epoch,
    validation_data=valid_generator,
    validation_steps=validation_steps).history

# Salvar o extrator de características
if not os.path.exists('/content/drive/MyDrive/ImgSim/'):
    os.mkdir('/content/drive/MyDrive/ImgSim/')

feature_extractor = tf.keras.Model(inputs=model.inputs, outputs=model.layers[-3].output)
feature_extractor.save('/content/drive/MyDrive/ImgSim/bit_feature_extractor', save_format='tf')

saved_model_path = '/content/drive/MyDrive/ImgSim/bit_model'
tf.saved_model.save(model, saved_model_path)

In [ ]:
import tensorflow as tf
from pathlib import Path
import numpy as np
import os
from tqdm import tqdm
tqdm.pandas()
img_paths = []
for path in Path('/content/Fashion_data/categories').rglob('*.jpg'):
  img_paths.append(path)
np.random.shuffle(img_paths)

def load_img(path):
  img = tf.io.read_file(path)
  img = tf.io.decode_jpeg(img, channels=3)
  img = tf.image.resize_with_pad(img, 224, 224)
  img = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
  return img

TRANSFER_LEARNING_FLAG = 1
if TRANSFER_LEARNING_FLAG:
  module = tf.keras.models.load_model('/content/drive/MyDrive/ImgSim/bit_feature_extractor')
else:
  module_handle = "https://tfhub.dev/google/bit/s-r50x3/ilsvrc2012_classification/1"
  module = hub.load(module_handle)

imgvec_path = '/content/img_vectors/'
Path(imgvec_path).mkdir(parents=True, exist_ok=True)

for filename in tqdm(img_paths[:5000]):
    img = load_img(str(filename))
    features = module(img)
    feature_set = np.squeeze(features)
    outfile_name = os.path.basename(filename).split('.')[0] + ".npz"
    out_path_file = os.path.join(imgvec_path, outfile_name)
    np.savetxt(out_path_file, feature_set, delimiter=',')

In [ ]:
import pandas as pd
import glob
import os
import numpy as np
from tqdm import tqdm
tqdm.pandas()
!pip install -q annoy
import json
from annoy import AnnoyIndex
from scipy import spatial
import pickle
from IPython.display import Image as dispImage
from IPython.display import display

# O 'styles.csv' precisa estar acessível, então ajuste o caminho se necessário.
# Exemplo: root_path = '/content/'
# styles = pd.read_csv(root_path+'styles.csv', error_bad_lines=False)
# styles['id'] = styles['id'].astype('str')
# styles.to_csv(root_path+'styles.csv', index=False)
# def match_id(fname):
#   return styles.index[styles.id==fname].values[0]

# Definir estruturas de dados
file_index_to_file_name = {}
file_index_to_file_vector = {}
file_index_to_product_id = {}

# Configurar Annoy
dims = 256
n_nearest_neighbors = 20
trees = 10000

# Ler todos os arquivos de vetores
allfiles = glob.glob('/content/img_vectors/*.npz')

# Criar e popular o índice Annoy
t = AnnoyIndex(dims, metric='angular')
for findex, fname in tqdm(enumerate(allfiles)):
    file_vector = np.loadtxt(fname)
    file_name = os.path.basename(fname).split('.')[0]
    file_index_to_file_name[findex] = file_name
    file_index_to_file_vector[findex] = file_vector
    t.add_item(findex, file_vector)

# Salvar o índice Annoy
annoy_model_path = '/content/drive/MyDrive/ImgSim/fashion_annoy.ann'
t.build(trees)
t.save(annoy_model_path)

# Salvar os metadados
with open('/content/drive/MyDrive/ImgSim/file_index_to_file_name.json', 'w') as f:
    json.dump(file_index_to_file_name, f)

with open('/content/drive/MyDrive/ImgSim/file_index_to_product_id.json', 'w') as f:
    json.dump(file_index_to_product_id, f)

# Carregar o modelo, o índice e os metadados
def load_recommendation_system(model_path, annoy_path, metadata_path):
    print("Loading feature extractor model...")
    extractor = tf.keras.models.load_model(model_path)

    print("Loading Annoy index...")
    dims = extractor.output_shape[1]
    annoy_index = AnnoyIndex(dims, 'angular')
    annoy_index.load(annoy_path)

    with open(metadata_path + 'file_index_to_file_name.json', 'r') as f:
        file_index_to_file_name = json.load(f)

    return extractor, annoy_index, file_index_to_file_name

# Função para encontrar e exibir imagens similares
def find_similar_images(image_path, extractor, annoy_index, file_index_to_file_name, n_results=5):
    print(f"Finding similar images for: {image_path}")

    # 1. Carregar e pré-processar a imagem de busca
    img = load_img(image_path)

    # 2. Extrair o vetor de características da imagem de busca
    search_vector = np.squeeze(extractor(img))

    # 3. Encontrar os vizinhos mais próximos no índice Annoy
    nearest_neighbors_ids = annoy_index.get_nns_by_vector(search_vector, n=n_results, search_k=-1)

    # Exibir a imagem de busca
    print("\n--- Imagem de Busca ---")
    display(dispImage(filename=image_path, width=200))

    # Exibir as imagens recomendadas
    print("\n--- Imagens Recomendadas ---")

    for neighbor_id in nearest_neighbors_ids:
        # Pular a própria imagem de busca
        if file_index_to_file_name[str(neighbor_id)] == os.path.basename(image_path).split('.')[0]:
            continue

        recommended_img_id = file_index_to_file_name[str(neighbor_id)]
        recommended_img_path = glob.glob(f'/content/Fashion_data/categories/**/{recommended_img_id}.jpg', recursive=True)[0]

        display(dispImage(filename=recommended_img_path, width=200))
        print(f"ID do produto: {recommended_img_id}")

# --- Exemplo de Uso do Sistema ---
# Caminhos salvos na seção 2
model_path = '/content/drive/MyDrive/ImgSim/bit_feature_extractor'
annoy_path = '/content/drive/MyDrive/ImgSim/fashion_annoy.ann'
metadata_path = '/content/drive/MyDrive/ImgSim/'

# Carregar o sistema de recomendação
feature_extractor, annoy_index, file_index_to_file_name = load_recommendation_system(model_path, annoy_path, metadata_path)

# Caminho para uma imagem de teste (substitua pelo seu próprio)
test_img = '/content/Fashion_data/categories/Accessories/1941.jpg'

# Encontrar e exibir as recomendações
find_similar_images(test_img, feature_extractor, annoy_index, file_index_to_file_name, n_results=5)